In [ ]:
#Wczytanie pliku .seq i odczytanie jego podstawowych informacji
from flirpy.io.seq import Seq

# Ścieżka do pliku .seq
file_path = "Sciezka"

try:
    # Wczytanie pliku .seq
    seq = Seq(file_path)
    
    # Info o pliku
    print(f"Liczba klatek w pliku: {len(seq)}")
    print(f"Rozmiar danych w pliku: {len(seq.seq_blob)} bajtów")
    
    # Info o pierwszej klatce
    if len(seq) > 0:
        first_frame = seq[0]
        print(f"Pierwsza klatka w formacie FFF: {type(first_frame)}")

    print("Plik został wczytany poprawnie.")
except Exception as e:
    print("Wystąpił błąd podczas wczytywania pliku:", e)


In [ ]:
#Zapis plików .seq w formacie tiff 16bit
# Automatyzacja dla wszystkich plików z folderu i tworzenie nowych folderów na klatki z poszczególnych plików .seq
import os
from flirpy.io.seq import Seq
from PIL import Image
import numpy as np

# Ścieżka do folderu z plikami .seq
input_folder = "Sciezka"

# Ścieżka do folderu wyjściowego
output_base_folder = "Sciezka"

# Tworzenie folderu głównego na klatki, jeśli nie istnieje
os.makedirs(output_base_folder, exist_ok=True)

try:
    # Iteracja przez wszystkie pliki .seq w folderze
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".seq"):
            # Pełna ścieżka do pliku .seq
            file_path = os.path.join(input_folder, file_name)
            
            # Tworzenie folderu dla aktualnego pliku .seq
            seq_folder_name = os.path.splitext(file_name)[0]  # Usuwanie rozszerzenia
            seq_output_folder = os.path.join(output_base_folder, seq_folder_name)
            os.makedirs(seq_output_folder, exist_ok=True)

            print(f"Przetwarzanie pliku: {file_name}")
            
            # Wczytanie pliku .seq
            seq = Seq(file_path)

            # Iteracja przez klatki i zapis jako TIFF 16-bit
            for i in range(len(seq)):
                frame = seq[i]

                # Pobranie obrazu w formie numpy array
                image_data = frame.get_image()  # Zwraca obraz w postaci numpy array

                # Normalizacja do 16-bit
                image_data = (image_data - image_data.min()) / (image_data.max() - image_data.min()) * 65535
                image_data = image_data.astype(np.uint16)

                # Tworzenie ścieżki dla pliku TIFF
                output_file = os.path.join(seq_output_folder, f"frame_{i:04d}.tiff")

                # Zapis obrazu jako TIFF 16-bit
                Image.fromarray(image_data).save(output_file, format="TIFF")

            print(f"Klatki z pliku {file_name} zostały zapisane w folderze: {seq_output_folder}")
except Exception as e:
    print("Wystąpił błąd:", e)


In [ ]:
#wykrywanie ROI dla tiff
import dlib
import cv2
import os
from PIL import Image
import numpy as np

# Ścieżka do folderu z klatkami TIFF
input_folder = "Sciezka"
output_folder = "Sciezka"

# Tworzenie folderu wyjściowego
os.makedirs(output_folder, exist_ok=True)

# detektor twarzy z Dlib
detector = dlib.get_frontal_face_detector()

# Minimalny akceptowalny rozmiar obszaru czoła
MIN_WIDTH = 50
MIN_HEIGHT = 20

def extract_forehead_with_dlib(image_path, output_path, frame_number):
    try:
        # Wczytanie obrazu w skali szarości (16-bitów)
        image = Image.open(image_path)
        image_16bit = np.array(image)

        # Dynamiczne skalowanie do 8-bitów
        min_val, max_val = np.min(image_16bit), np.max(image_16bit)
        image_8bit = ((image_16bit - min_val) / (max_val - min_val) * 255).astype(np.uint8)

        # Detekcja twarzy
        faces = detector(image_8bit)

        if len(faces) == 0:
            print(f"Nie wykryto twarzy dla klatki {frame_number}. Pomijam.")
            return

        # interesuje nas pierwsza wykryta twarz
        face = faces[0]
        x, y, w, h = face.left(), face.top(), face.width(), face.height()

        # Wyznaczenie obszaru czoła
        forehead_y_start = max(0, y - int(h * 0.25))  # Przesunięcie w górę o 25% wysokości twarzy
        forehead_y_end = y + int(h * 0.15)            # Górne 15% twarzy
        x_min = x
        x_max = x + w
        roi_forehead = image_16bit[forehead_y_start:forehead_y_end, x_min:x_max]

        # Walidacja rozmiaru ROI
        if roi_forehead.shape[0] < MIN_HEIGHT or roi_forehead.shape[1] < MIN_WIDTH:
            print(f"Pominięto ROI dla klatki {frame_number} (za mały obszar: {roi_forehead.shape}).")
            return

        # Zapis obrazu ROI jako TIFF
        output_file = os.path.join(output_path, f"forehead_frame_{frame_number:04d}.tiff")
        Image.fromarray(roi_forehead).save(output_file, format="TIFF", bits=16)

    except Exception as e:
        print(f"Błąd podczas przetwarzania klatki {frame_number}: {e}")


# Iteracja przez wszystkie foldery z klatkami
for folder_name in os.listdir(input_folder):
    folder_path = os.path.join(input_folder, folder_name)
    if os.path.isdir(folder_path):
        print(f"Aktualnie analizowany folder: {folder_name}")
        roi_output_folder = os.path.join(output_folder, folder_name)
        os.makedirs(roi_output_folder, exist_ok=True)

        for file_name in sorted(os.listdir(folder_path)):
            if file_name.endswith(".tiff"):
                file_path = os.path.join(folder_path, file_name)
                frame_number = int(os.path.splitext(file_name)[0].split("_")[-1])
                extract_forehead_with_dlib(file_path, roi_output_folder, frame_number)

print(f"Wszystkie obszary ROI (czoło) zostały zapisane w folderze: {output_folder}")


In [ ]:
# Ekstrakcja średnich wartości temperatur z ROI
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

# Ścieżka do folderu z wyznaczonymi ROI
roi_folder = "Sciezka"

# Lista do przechowywania wartości średnich temperatur
average_temperatures = []

# Funkcja do przetwarzania pojedynczego obrazu
def process_image(file_path):
    try:
        # Wczytaj obraz w skali szarości
        image = Image.open(file_path)
        image_data = np.array(image)

        if image_data.size == 0:
            print(f"Pominięto pusty obraz: {file_path}")
            return None

        # Normalizacja obrazu do zakresu [0, 1] (opcjonalna diagnostyka)
        normalized_image = (image_data - image_data.min()) / (image_data.max() - image_data.min())

        # Dynamiczne ustawienie progu na podstawie percentyla
        threshold_value = np.percentile(image_data, 95)

        # Zastosowanie maski do wykluczenia tła
        mask = image_data > threshold_value
        if np.sum(mask) == 0:  # Sprawdzenie, czy maska nie jest pusta
            print(f"Maska pusta dla obrazu: {file_path}")
            return None

        masked_image = np.where(mask, image_data, 0)

        # Zastosowanie filtra Gaussa dla redukcji szumu
        smoothed_image = gaussian_filter(masked_image, sigma=1.5)

        # Obliczanie średniej wartości temperatury z zamaskowanego obrazu
        avg_temp = np.mean(smoothed_image[smoothed_image > 0])  # Tylko niezerowe wartości

        return avg_temp, mask, smoothed_image

    except Exception as e:
        print(f"Błąd podczas przetwarzania obrazu {file_path}: {e}")
        return None

# Iteracja przez obrazy ROI w wybranym folderze
for file_name in sorted(os.listdir(roi_folder)):
    if file_name.endswith(".tiff"):
        file_path = os.path.join(roi_folder, file_name)
        result = process_image(file_path)
        
        if result is not None:
            avg_temp, mask, smoothed_image = result
            average_temperatures.append(avg_temp)

# Sprawdzenie zakresu temperatur dla diagnostyki
if average_temperatures:
    print(f"Minimalna wartość temperatury: {min(average_temperatures):.2f}")
    print(f"Maksymalna wartość temperatury: {max(average_temperatures):.2f}")

# Wizualizacja sygnału temperatury
plt.figure(figsize=(10, 5))
plt.plot(average_temperatures, label="Średnia temperatura ROI")
plt.xlabel("Numer klatki")
plt.ylabel("Średnia temperatura [jednostki obrazu]")
plt.title("Sygnał temperatury z ROI (po filtracji i maskowaniu)")
plt.legend()
plt.grid(True)
plt.show()

# Wizualizacja histogramu wartości średnich temperatur
plt.figure(figsize=(10, 5))
plt.hist(average_temperatures, bins=50, color='blue', alpha=0.7)
plt.xlabel("Średnia temperatura [jednostki obrazu]")
plt.ylabel("Liczba wystąpień")
plt.title("Histogram średnich temperatur z ROI")
plt.grid(True)
plt.show()

# debugowanie: Zapisanie ostatniej maski i przetworzonego obrazu
if 'mask' in locals() and 'smoothed_image' in locals():
    plt.figure(figsize=(12, 6))

    # Maska
    plt.subplot(1, 2, 1)
    plt.imshow(mask, cmap="gray")
    plt.title("Maska")
    plt.axis("off")

    # Obraz po maskowaniu
    plt.subplot(1, 2, 2)
    plt.imshow(smoothed_image, cmap="hot")
    plt.title("Obraz po zastosowaniu maski")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

# Opcjonalnie: Zapis sygnału do pliku
#np.savetxt("average_temperatures.txt", average_temperatures, fmt="%.6f")


In [ ]:
# Filtracja i normalizacja sygnału
from scipy.signal import butter, filtfilt
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft

# Funkcja do projektowania filtra Butterwortha
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs  # Częstotliwość Nyquista (połowa częstotliwości próbkowania)
    low = lowcut / nyquist  # Normalizacja dolnej granicy
    high = highcut / nyquist  # Normalizacja górnej granicy
    b, a = butter(order, [low, high], btype='band')  # Projektowanie filtra pasmowego
    return b, a

# Funkcja do filtrowania sygnału
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)  # Filtracja w przód i wstecz
    return y

# Funkcja do analizy FFT
def analyze_fft(signal, fs):

    n = len(signal)  # Liczba próbek
    freqs = np.fft.fftfreq(n, d=1/fs)  # Skala częstotliwości
    fft_values = fft(signal)  # Transformata FFT
    amplitudes = np.abs(fft_values)  # Amplitudy (moduł FFT)
    return freqs[:n//2], amplitudes[:n//2]  # Tylko dodatnie częstotliwości

# Normalizacja sygnału do zakresu -1 do 1
def normalize_signal_to_neg1_to_1(signal):

    return 2 * ((signal - np.min(signal)) / (np.max(signal) - np.min(signal))) - 1

# Stała wartość FPS kamery
fs = 30.0  # Liczba klatek na sekundę

# average_temperatures to lista średnich wartości temperatur
signal_raw = np.array(average_temperatures)

# Odejmowanie średniej sygnału (usunięcie trendu)
signal_raw_detrended = signal_raw - np.mean(signal_raw)

# Analiza FFT w celu dynamicznego dostosowania granic filtra
freqs, amps = analyze_fft(signal_raw_detrended, fs)

# Dynamiczne dostosowanie parametrów filtra
dominant_freq = freqs[np.argmax(amps)]
if 0.7 <= dominant_freq <= 3.5:  
    lowcut = max(0.5, dominant_freq - 0.3)  # Dolna granica pasma
    highcut = lowcut + 1.8  # Górna granica pasma
else:
    lowcut = 0.7  # Domyślna dolna granica
    highcut = 3.5  # Domyślna górna granica

print(f"Dostosowane parametry filtra: lowcut={lowcut:.2f}, highcut={highcut:.2f}")

# Filtracja sygnału
order = 5
filtered_signal = bandpass_filter(signal_raw_detrended, lowcut, highcut, fs, order)

# Normalizacja sygnału do zakresu -1 do 1
normalized_signal = normalize_signal_to_neg1_to_1(filtered_signal)

# Wizualizacja sygnałów
plt.figure(figsize=(12, 6))

# Sygnał surowy
plt.subplot(3, 1, 1)
plt.plot(signal_raw, label="Sygnał surowy", color="gray")
plt.xlabel("Numer klatki")
plt.ylabel("Średnia temperatura")
plt.title("Sygnał surowy")
plt.legend()
plt.grid(True)

# Sygnał po filtracji
plt.subplot(3, 1, 2)
plt.plot(filtered_signal, label=f"Sygnał po filtracji ({lowcut:.2f}–{highcut:.2f} Hz)", color="blue")
plt.xlabel("Numer klatki")
plt.ylabel("Temperatura")
plt.title("Sygnał po filtracji")
plt.legend()
plt.grid(True)

# Sygnał znormalizowany
plt.subplot(3, 1, 3)
plt.plot(normalized_signal, label="Sygnał znormalizowany (-1 do 1)", color="green")
plt.xlabel("Numer klatki")
plt.ylabel("Znormalizowana wartość")
plt.title("Sygnał znormalizowany")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Porównanie sygnałów dla pierwszych 200 próbek
plt.figure(figsize=(10, 5))
plt.plot(signal_raw[:200], label="Sygnał surowy (pierwsze 200 próbek)", color="gray")
plt.plot(filtered_signal[:200], label="Sygnał po filtracji (pierwsze 200 próbek)", color="blue")
plt.xlabel("Numer klatki")
plt.ylabel("Temperatura")
plt.title("Porównanie sygnału przed i po filtracji (początek)")
plt.legend()
plt.grid(True)
plt.show()

# FFT sygnału surowego przed filtracją
plt.figure(figsize=(10, 5))
plt.plot(freqs, amps, label="FFT sygnału surowego (po usunięciu trendu)", color="purple")
plt.xlabel("Częstotliwość (Hz)")
plt.ylabel("Amplituda")
plt.title("FFT sygnału surowego po usunięciu trendu")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
#PCA i ICA
from sklearn.decomposition import PCA, FastICA
import numpy as np
import matplotlib.pyplot as plt

# Funkcja do zastosowania PCA
def apply_pca(signal, n_components=1):

    pca = PCA(n_components=n_components)
    signal_reshaped = signal.reshape(-1, 1)  # Dopasowanie wymiarów do PCA
    principal_components = pca.fit_transform(signal_reshaped)
    return principal_components.flatten(), pca

# Funkcja do zastosowania ICA
def apply_ica(signal, n_components=1):

    ica = FastICA(n_components=n_components, random_state=42)
    signal_reshaped = signal.reshape(-1, 1)  # Dopasowanie wymiarów do ICA
    independent_components = ica.fit_transform(signal_reshaped)
    return independent_components.flatten(), ica

# Zastosowanie PCA
pca_signal, pca_model = apply_pca(normalized_signal, n_components=1)

# Zastosowanie ICA
ica_signal, ica_model = apply_ica(normalized_signal, n_components=1)

# Wizualizacja wyników PCA i ICA
plt.figure(figsize=(12, 6))

# Sygnał po PCA
plt.subplot(2, 1, 1)
plt.plot(pca_signal, label="Sygnał po PCA", color="orange")
plt.xlabel("Numer klatki")
plt.ylabel("Znormalizowana wartość")
plt.title("Sygnał po PCA")
plt.legend()
plt.grid(True)

# Sygnał po ICA
plt.subplot(2, 1, 2)
plt.plot(ica_signal, label="Sygnał po ICA", color="purple")
plt.xlabel("Numer klatki")
plt.ylabel("Znormalizowana wartość")
plt.title("Sygnał po ICA")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Opcjonalnie: Zapis wyników do plików
#np.savetxt("pca_signal.txt", pca_signal, fmt="%.6f")
#np.savetxt("ica_signal.txt", ica_signal, fmt="%.6f")

plt.figure(figsize=(12, 6))
plt.plot(normalized_signal, label="Sygnał znormalizowany", color="green", alpha=0.6)
plt.plot(pca_signal, label="Sygnał po PCA", color="orange", alpha=0.8)
plt.plot(ica_signal, label="Sygnał po ICA", color="purple", alpha=0.8)
plt.xlabel("Numer klatki")
plt.ylabel("Znormalizowana wartość")
plt.title("Porównanie sygnału znormalizowanego, PCA i ICA")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Analiza FFT
from scipy.fftpack import fft
import numpy as np
import matplotlib.pyplot as plt

# Funkcja do analizy FFT
def analyze_fft(signal, fs):

    n = len(signal)  # Liczba próbek
    freqs = np.fft.fftfreq(n, d=1/fs)  # Skala częstotliwości
    fft_values = fft(signal)  # Transformata FFT
    amplitudes = np.abs(fft_values)  # Amplitudy (moduł FFT)
    
    # Zwrot tylko dla dodatnich częstotliwości
    pos_freqs = freqs[:n//2]
    pos_amplitudes = amplitudes[:n//2]
    return pos_freqs, pos_amplitudes

# Parametry
fs = 30.0  # Częstotliwość próbkowania (liczba klatek na sekundę - dostosuj do swoich danych)

# Detrendowanie sygnału
ica_signal_detrended = ica_signal - np.mean(ica_signal)

# Analiza FFT dla sygnału po ICA
freqs_ica, amplitudes_ica = analyze_fft(ica_signal_detrended, fs)

# Filtrowanie częstotliwości w odpowiednim zakresie
valid_indices = (freqs_ica >= 0.7) & (freqs_ica <= 3.5)
freqs_ica = freqs_ica[valid_indices]
amplitudes_ica = amplitudes_ica[valid_indices]

# Znalezienie dominującej częstotliwości
dominant_freq_ica = freqs_ica[np.argmax(amplitudes_ica)]
heart_rate_ica = dominant_freq_ica * 60  # Przeliczenie na BPM (uderzenia na minutę)

# Wizualizacja wyniku FFT
plt.figure(figsize=(10, 5))
plt.plot(freqs_ica, amplitudes_ica, label="FFT sygnału po ICA", color="purple")
plt.axvline(dominant_freq_ica, color="red", linestyle="--", label=f"Dominująca: {dominant_freq_ica:.2f} Hz")
plt.text(dominant_freq_ica, max(amplitudes_ica), f"{heart_rate_ica:.2f} BPM", color="red")
plt.xlabel("Częstotliwość (Hz)")
plt.ylabel("Amplituda")
plt.title("Analiza FFT - Sygnał po ICA")
plt.legend()
plt.grid(True)
plt.show()

# Wyświetlenie wyniku
print(f"Dominująca częstotliwość: {dominant_freq_ica:.2f} Hz")
print(f"Tętno: {heart_rate_ica:.2f} BPM")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks

# Funkcja do projektowania filtra Butterwortha (dla wygładzania sygnału)
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Funkcja do filtrowania sygnału
def smooth_signal(signal, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order)
    smoothed_signal = filtfilt(b, a, signal)
    return smoothed_signal

# Rekonstrukcja sygnału na podstawie ICA
time = np.arange(len(ica_signal)) / fs  # Czas w sekundach (przy częstotliwości próbkowania fs)
reconstructed_signal = ica_signal  # Użyjemy sygnału po ICA

# Wygładzenie sygnału (opcjonalne filtrowanie)
reconstructed_signal_smoothed = smooth_signal(reconstructed_signal, 0.7, 3.5, fs)

# Normalizacja do zakresu [0, 1]
reconstructed_signal_normalized = (reconstructed_signal_smoothed - np.min(reconstructed_signal_smoothed)) / \
                                   (np.max(reconstructed_signal_smoothed) - np.min(reconstructed_signal_smoothed))

# Detekcja szczytów (uderzenia serca)
peaks, _ = find_peaks(reconstructed_signal_normalized, height=0.5, distance=fs//2)

peak_times = peaks / fs

# Wizualizacja sygnału tętna z oznaczeniem szczytów
plt.figure(figsize=(12, 6))
plt.plot(time, reconstructed_signal_normalized, label="Zrekonstruowany sygnał tętna", color="blue")
plt.scatter(peak_times, reconstructed_signal_normalized[peaks], color="red", label="Szczyty (uderzenia serca)", zorder=5)
plt.axhline(y=0.5, color="gray", linestyle="--", alpha=0.7, label="Próg detekcji szczytów")  # Linia progu
plt.xlabel("Czas (s)")
plt.ylabel("Znormalizowana wartość")
plt.title("Wykres tętna w czasie z oznaczeniem szczytów")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
import numpy as np

# Dane referencyjne
ref_pulses = []

# Puls obliczony przez program
avg_pulse_program = heart_rate_ica

# Obliczenie średniej z referencyjnych danych
avg_ref_pulse = np.mean(ref_pulses)

# Obliczenie MAE
mae = np.abs(avg_ref_pulse - avg_pulse_program)

# Wyniki
print(f"Średni puls referencyjny: {avg_ref_pulse:.2f}")
print(f"Średni puls obliczony przez program: {avg_pulse_program:.2f}")
print(f"Błąd średni bezwzględny (MAE): {mae:.2f}")
